# 621. Task Scheduler

[leetcode](https://leetcode.com/problems/task-scheduler/)

Given a characters array tasks, representing the tasks a CPU needs to do, where each letter represents a different task. Tasks could be done in any order. Each task is done in one unit of time. For each unit of time, the CPU could complete either one task or just be idle.

However, there is a non-negative integer n that represents the cooldown period between two same tasks (the same letter in the array), that is that there must be at least n units of time between any two same tasks.

Return the least number of units of times that the CPU will take to finish all the given tasks.

# Reasoning

[neetcodevideo](https://www.youtube.com/watch?v=s8p8ukTyA2I&ab_channel=NeetCode)

Given an [A,A,A,B,B,B], the suggested order of processing is [A,B,_,A,B,_], where '_' is the idle time, since we cannot process another task of the same kind untill we wait two time nits. We used one time unit to process 'B' but we need to wait another one.  

Consider an input AAABBCC, and n=1. First we collect the number of the same characters. We start processing with the most frequenct character, so that we have less of those, than we need to process the second most requent, which is not the same (so we avoid the cooldown time). In the end we arrive to the [ABCABCA], which minimizes the cooldown.  

We can get the list of most frequent tasks with `max heap` structure. The time complexity of getting the most frequenct log(N=26), where 26 is the number of unique values. However, we need to also go through all characters to get their frequency, so the overall time complexity is O(n). 

Memory complexity is also O(N) as we would need t oconstruct the map of number of occurancies. 

The _maxheap_ is used for integers, for the number of occurancies of different characters. In python we only have _min heap_ so we use -1 for all counts.  

Each time we are processing a task we do _heappop_, decriment the value by one (we processed one task). We __do not add the decrimented count back__. We have to waid the _cooldown_. This is done via a separate datastructure.

At the same time we add the character that we just processed to the `queue` data structure to keep track on the cooldown time. In the `queue` we store two values: the current count for the task and the time when this task is available again. The time is computed as 0 from the start and each time _a task_ is executed, the time advances. So, in the queue entry we have [[N_c, T_c]], numer of task of this type (character) remaining, time when we can execute this task again. 

Once the current time = T_c, we do _heappush_ of the current remaining tasks of this kind to the heap. 

So the algorithm:
- Construct initial min heap of tasks and occurances (numers), set time to 0
- Pop the most frequent (largest N) task, increment the time t+=1
- Decriment the number by 1, N-1, and add [N-1, t+tcool] to the qeue.
- If current time = t+tcool in the queue, we add it back to the heap ( the N )
- If task N = 0, we do not process it and we do not need to increment time

Poppping and pusing to the max heap is O(25)
9

In [ ]:
from typing import List
from collections import Counter, deque
import heapq
class Solution:
    def leastInterval(self, tasks: List[str], n: int) -> int:
        """  Solution using max heap and queue data structures
         Each task takes 1 unit of time.
          Goal: minimize idle time
          The time complexity of the while solution is O(n * m), where 
          m is the idle time ()
        """
        # create a hash map
        count = Counter(tasks)
        # create max heap using counter (get -1 for max heap in python)
        maxHeap = [-cnt for cnt in count.values()]
        # creat min heap (order the array as if it is max heap) O(n) operation
        heapq.heapify(maxHeap)
        time = 0
        # init the queue data structure
        q = deque() # pars of [-cnt, idelTime] (at what time we can add it back to heap)

        while maxHeap or q: # while any of them is not empty, we have tasks to process
            time +=1 
            if maxHeap:
                val = heapq.heappop(maxHeap)
                cnt = 1 + val # as we have negative values
                if cnt:
                    q.append([cnt,time+n]) # add current count and time
            # check if qeue is not empty and the idle time in the first element is reached
            if q and q[0][1] == time: 
                # remove the N of tasks from the qeue and push it to back to the heap
                pair = q.popleft()  
                heapq.heappush(maxHeap, pair[0]) # only push the counts for task remaining
        return time

